# Make Dataset

## Imports

In [1]:
from PlantReactivityAnalysis.data.wav_data_reader import WavDataReader
from PlantReactivityAnalysis.data.signal_dataset import SignalDataset
from PlantReactivityAnalysis.features.wav_feature_extractor import WavFeatureExtractor
from PlantReactivityAnalysis.features.features_dataset import FeaturesDataset
import PlantReactivityAnalysis.data.preparation_eurythmy_data as ped

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Reader

In [3]:
wav_folder= r"..\data\raw\wav_files"
#wav_folder= r"..\data\interim\testing"

In [3]:
# Initialize the reader with the folder of wavs
reader= WavDataReader(folder= wav_folder, sample_rate= 10000)

# Get the signals and the keys from the reader
signals, ids= reader.get_values_and_keys()

# Get the measurement labels from the keys of the files
meas_df= ped.return_meas_labels_by_keys(ids)

Total WAV files read: 625


In [4]:
meas_df.head()

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy
0,1,1,2023-04-29,salad,1,1
1,2,1,2023-04-29,salad,1,1
2,3,1,2023-04-29,salad,1,1
3,4,1,2023-04-29,salad,2,0
4,5,1,2023-04-29,salad,2,0


## Signal Dataset

### Raw

In [5]:
# Initialize SignalDataset with the signals and features/labels
signal_dataset= SignalDataset(signals= signals, features= meas_df)

In [6]:
signal_dataset.features.shape

(625, 6)

In [7]:
signal_dataset.features.head()

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy
0,1,1,2023-04-29,salad,1,1
1,2,1,2023-04-29,salad,1,1
2,3,1,2023-04-29,salad,1,1
3,4,1,2023-04-29,salad,2,0
4,5,1,2023-04-29,salad,2,0


In [8]:
# Save and Load the dataset
raw_signal_dataset_path= r"..\data\raw\raw_signal_dataset.pkl"
signal_dataset.save(raw_signal_dataset_path)
signal_dataset= SignalDataset.load(raw_signal_dataset_path)

### Normalized

In [9]:
# Standardize Signals using Zscore
signal_dataset.standardize_signals("zscore")

In [10]:
signal_dataset.features.shape

(625, 6)

In [11]:
# Save the dataset
norm_signal_dataset_path= r"..\data\interim\norm_signal_dataset.pkl"
signal_dataset.save(norm_signal_dataset_path)

### Segmented by Letters (raw)

In [12]:
# Load raw dataset
signal_dataset= SignalDataset.load(raw_signal_dataset_path)

In [13]:
# Get Letter segments from raw data
letter_dictionary= ped.return_letter_dictionary(indexes= signal_dataset.features['id_measurement'].tolist())

In [14]:
# Segment the dataset
signal_dataset.segment_signals_by_dict('id_measurement', letter_dictionary, 'eurythmy_letter')

In [15]:
signal_dataset.features.shape

(8390, 7)

In [16]:
signal_dataset.features.head()

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy,eurythmy_letter
0,1,1,2023-04-29,salad,1,1,A1
1,1,1,2023-04-29,salad,1,1,G1
2,1,1,2023-04-29,salad,1,1,D1
3,1,1,2023-04-29,salad,1,1,A2
4,1,1,2023-04-29,salad,1,1,G2


In [4]:
raw_letters_signal_dataset_path= r"..\data\raw\raw_letters_signal_dataset.pkl"

In [18]:
# Save the dataset
signal_dataset.save(raw_letters_signal_dataset_path)

### Segmented by Letters (normalized)

In [19]:
# Load normalized dataset
signal_dataset= SignalDataset.load(norm_signal_dataset_path)

In [20]:
# Get Letter segments from raw data
letter_dictionary= ped.return_letter_dictionary(indexes= signal_dataset.features['id_measurement'].tolist())

In [21]:
# Segment the dataset
signal_dataset.segment_signals_by_dict('id_measurement', letter_dictionary, 'eurythmy_letter')

In [5]:
norm_letters_signal_dataset_path= r"..\data\interim\norm_letters_signal_dataset.pkl"

In [23]:
# Save the dataset
signal_dataset.save(norm_letters_signal_dataset_path)

### Segmented in 1s (raw)

In [24]:
# Load raw dataset
signal_dataset= SignalDataset.load(raw_signal_dataset_path)

In [25]:
# Segment Signals in 1sec segments
signal_dataset.segment_signals_by_duration(segment_duration= 1)

In [26]:
#Add eurythmy letter data to the features
df= ped.add_meas_letters(signal_dataset.features)

In [27]:
signal_dataset.features.shape

(148682, 8)

In [28]:
signal_dataset.features.iloc[15:20]

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy,initial_second,eurythmy_letter
15,1,1,2023-04-29,salad,1,1,15.0,None
16,1,1,2023-04-29,salad,1,1,16.0,None
17,1,1,2023-04-29,salad,1,1,17.0,A1
18,1,1,2023-04-29,salad,1,1,18.0,A1
19,1,1,2023-04-29,salad,1,1,19.0,A1


In [14]:
# Save the dataset
raw_1s_signal_dataset_path= r"..\data\raw\raw_1s_signal_dataset.pkl"
signal_dataset.save(raw_1s_signal_dataset_path)

### Segmented in 1s (normalized)

In [30]:
# Load normalized dataset
signal_dataset= SignalDataset.load(norm_signal_dataset_path)

In [31]:
# Segment Signals in 1sec segments
signal_dataset.segment_signals_by_duration(segment_duration=1)

In [32]:
#Add eurythmy letter data to the features
df= ped.add_meas_letters(signal_dataset.features)

In [33]:
# Save the dataset
norm_1s_signal_dataset_path= r"..\data\interim\norm_1s_signal_dataset.pkl"
signal_dataset.save(norm_1s_signal_dataset_path)

## Features Dataset

In [6]:
window_size= 2
hop_length= 2

feature_extractor= WavFeatureExtractor(sample_rate= 10000, lib_mfccs= True, pyau_mfccs= True, temporal= True,
                                       statistical= True, window_size= window_size, hop_length= hop_length)

### Normalized letters

In [7]:
# Load dataset
signal_dataset= SignalDataset.load(norm_letters_signal_dataset_path)

In [ ]:
%%time

# Extract features and store in dataset
feat_dataset= FeaturesDataset.from_signal_dataset(signal_dataset, feature_extractor)

C:\Users\alvar\anaconda3\envs\Plant-Reactivity-Analysis\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=20000 is too large for input signal of length=16940
  warnings.warn(
C:\Users\alvar\Documents\GitHub\Plant-Reactivity-Analysis\src\PlantReactivityAnalysis\features\wav_feature_extractor.py:297: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  feature_values.append(func(waveform_data))
C:\Users\alvar\anaconda3\envs\Plant-Reactivity-Analysis\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=20000 is too large for input signal of length=3860
  warnings.warn(
C:\Users\alvar\anaconda3\envs\Plant-Reactivity-Analysis\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=20000 is too large for input signal of length=18100
  warnings.warn(
C:\Users\alvar\anaconda3\envs\Plant-Reactivity-Analysis\lib\site-packages\librosa\co

In [ ]:
feat_dataset.features.shape

In [ ]:
feat_dataset.features.head()

In [ ]:
# Save the dataset
feat_norm_letters_dataset_path= r"..\data\processed\feat_norm_letters_" + str(window_size) + "_" + str(hop_length) + "_dataset.pkl"
feat_dataset.save(feat_norm_letters_dataset_path)

### Letters

In [ ]:
# Load dataset
signal_dataset= SignalDataset.load(raw_letters_signal_dataset_path)

In [ ]:
%%time

# Extract features and store in dataset
feat_dataset= FeaturesDataset.from_signal_dataset(signal_dataset, feature_extractor)

In [ ]:
# Save the dataset
feat_raw_letters_dataset_path= r"..\data\processed\feat_raw_letters_" + str(window_size) + "_" + str(hop_length) + "_dataset.pkl"
feat_dataset.save(feat_raw_letters_dataset_path)